In [39]:
import os
import logging
from typing import Dict
import xarray as xr
import numpy as np
import yaml
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"


def reset_chunk_encoding(da: xr.Dataset) -> xr.Dataset:
    """resets previous chunk encoding on concatenated zarr stores

    :param da: Dataset
    :return: Dataset with reset chunk encoding
    """
    for x in da.coords:
        if "chunks" in da[x].encoding.keys():
            logging.info(f"deleting chunk encoding for {x}...")
            del da[x].encoding["chunks"]
        if da.coords[x].dtype == object:
            da[x].encoding.clear()

    for var in da:
        if "chunks" in da[var].encoding.keys():
            logging.info(f"deleting chunk encoding for {var}...")
            del da[var].encoding["chunks"]
        if da[x].dtype == object:
            da[x].encoding.clear()
    return da


def find_nearest_idx(
    arr1: np.ndarray, arr2: np.ndarray, val1: float, val2: float,
) -> int:
    """Find first nearest index for a given tolerance for two arrays and 2 values

    :param arr1: first array
    :param arr2: second arrat
    :param val1: value to find in first array
    :param val2: value to find in second array
    :return: index as int
    """
    try:
        idx = (np.abs(arr1 - val1) + np.abs(arr2 - val2)).argmin()
    except:
        idx = 0
    return idx


def parse_yaml(yaml_path: str) -> Dict:
    """Helper function to open config yaml files

    :param yaml_path: path to config yaml
    :return: dictionary of meta-data for group name
    """
    with open(yaml_path, "r") as yaml_file:
        obs_dic = yaml.safe_load(yaml_file)
    return obs_dic

# Load config and inspect contents
CONFIG_DIC = parse_yaml("../config.yaml")
CONFIG_DIC

{'expver': 'i6aj',
 'start_date': '20190101',
 'end_date': '20191201',
 'climfile': '/home/daep/projects/ec_land_db/ec_land_db/scratch/surfclim_399_l4',
 'output_dir': '/ec/res4/scratch/daep/ec_training_db_out/',
 'grid': 'O400',
 'time_chunk_size': 10}

In [3]:
from dask_jobqueue import SLURMCluster
from distributed import Client

cluster = SLURMCluster(
    # cores=8,  # each worker will have 4 threads
    # processes=1,
    cores=4,  # each worker will have 4 threads
    processes=1,  #48,
    #memory="120GB",  # each worker will have 16GB memory
    memory="24GB",  # each worker will have 16GB memory
    #interface="ib0",  # makes sure workers can communicate with eachother
    walltime="01:30:00",
    job_extra_directives=['--qos=nf', 
                          '--output=logfiles/dask.%j.out',
                          '--error=logfiles/dask.%j.out'],  # spin up each worker on "nf" queue
    #job_extra_directives=['--qos=np'],  # spin up each worker on "nf" queue
    job_script_prologue=['module load conda; conda activate ec_land_db; export HDF5_USE_FILE_LOCKING=FALSE'],  # ensure correct python environment used
)

In [4]:
# See job script that will be submitted by Dask cluster
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=23G
#SBATCH -t 01:30:00
#SBATCH --qos=nf
#SBATCH --output=logfiles/dask.%j.out
#SBATCH --error=logfiles/dask.%j.out
module load conda; conda activate ec_land_db; export HDF5_USE_FILE_LOCKING=FALSE
/perm/daep/conda/envs/ec_land_db/bin/python3.8 -m distributed.cli.dask_worker tcp://10.100.64.204:41785 --nthreads 4 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60



In [3]:
# See job script that will be submitted by Dask cluster
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=23G
#SBATCH -t 01:30:00
#SBATCH --qos=nf
module load conda; conda activate dask-slurm; export HDF5_USE_FILE_LOCKING=FALSE
/perm/daep/conda/envs/dask-slurm/bin/python3.8 -m distributed.cli.dask_worker tcp://10.100.192.102:36217 --nthreads 4 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60



In [ ]:
client.shutdown()

In [5]:
# scale cluse to desired number of workers
cluster.scale(jobs=10)

In [12]:
# Watch the dask-workers spinning up
# Wait until dask works have started running
!squeue -u $USER

    JOBID       NAME  USER   QOS    STATE       TIME TIME_LIMIT NODES      FEATURES NODELIST(REASON)
 10698420 dask-worke  daep    nf  RUNNING       2:08    1:30:00     1        (null) aa6-176
 10698421 dask-worke  daep    nf  RUNNING       2:08    1:30:00     1        (null) aa6-176
 10698422 dask-worke  daep    nf  RUNNING       2:08    1:30:00     1        (null) aa6-172
 10698425 dask-worke  daep    nf  RUNNING       2:08    1:30:00     1        (null) aa6-173
 10698412 dask-worke  daep    nf  RUNNING       2:18    1:30:00     1        (null) aa6-108
 10698413 dask-worke  daep    nf  RUNNING       2:18    1:30:00     1        (null) aa6-185
 10698415 dask-worke  daep    nf  RUNNING       2:18    1:30:00     1        (null) aa6-108
 10698416 dask-worke  daep    nf  RUNNING       2:18    1:30:00     1        (null) aa6-185
 10698418 dask-worke  daep    nf  RUNNING       2:18    1:30:00     1        (null) aa6-189
 10698419 dask-worke  daep    nf  RUNNING       2:18    1:30:00     1  

In [13]:
# Connect our local client to the dask-workers we have spun up
client = Client(cluster)
client

<Client: 'tcp://10.100.64.204:41785' processes=10 threads=40, memory=223.50 GiB>

In [17]:
damet = xr.open_mfdataset("/ec/res4/scratch/daep/ec_training_db_out/ecland*.zarr", engine="zarr", parallel=True,)
damet = damet.rename({var: f"met_{var.lower()}" for var in damet.data_vars})
damet

<xarray.Dataset>
Dimensions:     (time: 4384, x: 187070)
Coordinates:
    lat         (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon         (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time        (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x           (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables:
    met_ctpf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_snowf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_tair    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_wind_e  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_wind_n  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

In [18]:
damod = xr.open_mfdataset("/ec/res4/scratch/daep/ec_training_db_out/i6aj*.zarr", engine="zarr", parallel=True,)
for var in list(damod.data_vars.keys()):
    if var[0] == "0":
        damod = damod.rename({var: var[1:]})
damod

<xarray.Dataset>
Dimensions:      (time: 4384, x: 187070)
Coordinates:
    lat          (x) float64 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float64 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/42)
    clim_CLAKE   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_Ctype   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cu      (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cvh     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cvl     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_geopot  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2023-10-25T21:05 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [19]:
da_train = reset_chunk_encoding(xr.merge([damet, damod.drop(["lat", "lon"])]))
list(da_train.data_vars.keys())

['met_ctpf',
 'met_lwdown',
 'met_psurf',
 'met_qair',
 'met_rainf',
 'met_swdown',
 'met_snowf',
 'met_tair',
 'met_wind_e',
 'met_wind_n',
 'clim_CLAKE',
 'clim_Ctype',
 'clim_cu',
 'clim_cvh',
 'clim_cvl',
 'clim_geopot',
 'clim_sdfor',
 'clim_sdor',
 'clim_sotype',
 'clim_tvh',
 'clim_tvl',
 'clim_z0m',
 'aco2gpp',
 'd2m',
 'dis',
 'e',
 'fal',
 'fldsto',
 'lai_hv',
 'lai_lv',
 'rsn',
 'rsn_l1',
 'rsn_l2',
 'sd',
 'sd_l1',
 'sd_l2',
 'skt',
 'snowc',
 'sro',
 'ssro',
 'stl1',
 'stl2',
 'stl3',
 'stl4',
 'swvl1',
 'swvl2',
 'swvl3',
 'swvl4',
 't2m',
 'tsn',
 'tsn_l1',
 'tsn_l2']

In [31]:
feats = [
 'met_ctpf',
 'met_lwdown',
 'met_psurf',
 'met_qair',
 'met_rainf',
 'met_swdown',
 'met_snowf',
 'met_tair',
 'met_wind_e',
 'met_wind_n',
 'clim_CLAKE',
 'clim_Ctype',
 'clim_cu',
 'clim_cvh',
 'clim_cvl',
 'clim_geopot',
 'clim_sdfor',
 'clim_sdor',
 'clim_sotype',
 'clim_tvh',
 'clim_tvl',
 'clim_z0m',
 'lai_hv',
 'lai_lv',
 'aco2gpp',
 # 'd2m',
 # 'dis',
 # 'e',
 # 'fal',
 # 'fldsto',
 'rsn',
 # 'rsn_l1',
 # 'rsn_l2',
 'sd',
 # 'sd_l1',
 # 'sd_l2',
 'skt',
 'snowc',
 'sro',
 'ssro',
 'stl1',
 'stl2',
 'stl3',
 # 'stl4',
 'swvl1',
 'swvl2',
 'swvl3',
 # 'swvl4',
 # 't2m',
 # 'tsn',
 # 'tsn_l1',
 # 'tsn_l2'
]

In [21]:
da_train

<xarray.Dataset>
Dimensions:      (time: 4384, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/52)
    met_ctpf     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

In [28]:
da_train.chunk({"time": CONFIG_DIC["time_chunk_size"]}).to_zarr(f"/ec/res4/scratch/daep/ec_training_db/ecland_{CONFIG_DIC["expver"]}_2020_2022.zarr", consolidated=True)

INFO:distributed.utils_perf:full garbage collection released 11.75 MiB from 280353 reference cycles (threshold: 9.54 MiB)
INFO:distributed.core:Event loop was unresponsive in Scheduler for 6.52s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.


In [31]:
da_test = xr.open_zarr("/ec/res4/scratch/daep/ec_training_db/ecland_cy49r2_2020_2022.zarr")
da_test

<xarray.Dataset>
Dimensions:      (time: 4384, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/52)
    aco2gpp      (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_CLAKE   (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_Ctype   (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cu      (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cvh     (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cvl     (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF